## Tutorial: Using pd_exp to create experiments with Prisoner's Dilemma Tournaments
This notebook walks through two different experimental setups to showcase the code functionality of the `pd_exp` module.

In [1]:
# Local modules to import
import pd_exp
from helper_funcs import partitions
from settings import player_names, stag

# Other Python modules used
import pandas as pd
from pathlib import Path
import time, json

<generator object partitions at 0x7fa8ad4f37d8>


`pd_exp` was designed to collect data from multiple groups of round-robin tournaments. To keep vocabulary consistent, let me introduce some project terminology:
- _Supergame_: a single PD instance between two players that consists of one or more rounds of play.
- _Tournament_: a round-robin tournament between a select group of players. This concept can also be referred to as a team.
- _System_: a collection of one or more teams.

In the code below, I setup data collection for two different experiments. In experiment 1, we consider how measured outcomes differ within one team of deterministic players in two different game variations: classic PD and Stag Hunt. In experiment 2, we expand on experiment 1 to consider multiple different systems, where each system is comprised of two teams.

#### __Team Metrics__
To measure team outcomes, we use the following metrics: 1) minimum of normalized player scores, 2) average of normalized player scores, and 3) average normalized CC distribution.

##### __The Minumum (Min Score) and Average (Avg Score) of Normalized Player Scores__
The normalized score for a player, $j$, is computed as 
$$
S_j = \sum_{i\neq j} \frac{\text{Score-per-turn against opponent } i}{\text{Total opponents}} = \frac{1}{\text{Total opponents}} \sum_{i\neq j} \frac{S_{ji}}{T_{ji}},
$$
where $S_{ji}$ is player $j$'s score against player $i$ and $T_{ji}$ is the total turns played between player $j$ and $i$.

The minimum and average values across all normalized player scores are taken as aggregate group measures to reflect group welfare.

##### __The New Average score
The new average score is computed as 

$$
\frac{\sum_{j=1}^N \sum_{i \neq j} {S_{ji}}}{\sum_{j=1}^N \sum_{i \neq j} {T_{ji}} / 2}
$$



##### __The Average Normalized CC Distribution (Avg CC Dist)__
The normalized CC distribution for a player represents the proportion of turns where they and their opponent chose to cooperate (CC) out of the total turns given. The average across each player's normalized CC distribution is used as an aggregate measure for overall group cooperation. The average normalized CC distribution is computed as 

$$
\frac{1}{N} \sum_{j=1}^N \frac{1}{N-1} \sum_{i \neq j} \frac{CC \text{ count against player } i}{\text{total turns against player }i}
$$

where $N$ is the total number of players in a model, $j$ is a given player, and $i$ is every other player.

##### __The New Average Normalized CC Distribution (Avg CC Dist)__
The normalized CC distribution for a player represents the proportion of turns where they and their opponent chose to cooperate (CC) out of the total turns given. The average across each player's normalized CC distribution is used as an aggregate measure for overall group cooperation. The average normalized CC distribution is computed as 

$$
\frac{\sum_{j=1}^N \sum_{i \neq j} {C_{ji}}}{\sum_{j=1}^N \sum_{i \neq j} {T_{ji}}}
$$

where $N$ is the total number of players in a model, $j$ is a given player, and $i$ is every other player.


#### __System Metrics__
To measure system outcomes, we extend the above metrics and use six different measures:
1. System Minimum Score (SYS MIN)
2. System Average Score (SYS AVG)
3. Minimum of Team Average Scores (Min of Team Avgs)
4. Average of Team Minimum Scores (Avg of Team Mins)
5. System CC Distribution Average (SYS CC Dist AVG)
6. System CC Distribution Average (SYS CC Dist MIN)

### Experiment 1: Comparing team performance across classic PD and Stag Hunt

In [2]:
### For experiment 1
# Since this simulation gathers data for systems, I coded systems as ordered-lists of 
# teams. To implement ordered-lists of teams I nested player_names within closed-brackets and then a tuple to create an ordered-list.
# The simulation code allows for processing numerous systems, so I enclose the tournament within another pair
# of closed-brackets and then a tuple.

part_list = tuple([tuple([player_names])]) # 'part' is shorthand for partition

In [3]:
print(part_list)

((['Bitter Cooperator', 'Cooperator', 'Fourteen Coop', 'Grim Trigger', 'Thirteen Coop', 'Tit For Tat', 'Win-Stay Lose-Shift', 'Defector', 'Fourteen Defect', 'Stubborn Defect', 'Suspicious Tit For Tat', 'Sucker Defect', 'Two Defect', 'Win-Shift Lose-Stay'],),)


In [4]:
print('Total count of systems: ',len(part_list))  #Should be 5775 for n=12 and k=4
start_time = time.time()

print('Instantiate PD Experiments with different game types')
# To instantiate a PD Experiment, I use the PDExp constructor with at partition list and game-type, which is 
# by default the classic PD)
classic_pdExp = pd_exp.PdExp(part_list)
stag_pdExp = pd_exp.PdExp(part_list,game_type=stag)


print('Running experiments and computing data')
# To generate the simulation data and compute the group metrics, I use the run_experiments() method.
classic_pdExp.run_experiments()
stag_pdExp.run_experiments()


print('Saving experiment data')
### For experiment 1
path = 'Data/Experiment1/'
classic_pdExp.save_data(path, 'ClassicPD_test_DELETE')  # CHANGE file_name string when necessary
stag_pdExp.save_data(path, 'StagHunt_test_DELETE')  # CHANGE file_name string when necessary

print("--- %s seconds ---" % (time.time() - start_time))

Total count of systems:  1
Instantiate PD Experiments with different game types
Running experiments and computing data
partition list:  (['Bitter Cooperator', 'Cooperator', 'Fourteen Coop', 'Grim Trigger', 'Thirteen Coop', 'Tit For Tat', 'Win-Stay Lose-Shift', 'Defector', 'Fourteen Defect', 'Stubborn Defect', 'Suspicious Tit For Tat', 'Sucker Defect', 'Two Defect', 'Win-Shift Lose-Stay'],)
Instantiating tournament object with these players:  Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player
Analysing: 100%|██████████| 25/25 [00:00<00:00, 40.93it/s]

Match length for each user against other users:  [[[3.0, 4.0, 8.0, 21.0, 12.0, 20.0, 30.0, 2.0, 4.0, 6.0, 2.0, 10.0, 16.0, 3.0], [

TypeError: 'tuple' object does not support item assignment

### Experiment 1: Data Comparison Across Games

In [5]:
# Retrieve saved data
classic_dataf1 = pd.read_csv('Data/Experiment1/ClassicPD_test_DELETE_RPST_3_1_0_5.csv', index_col=0)
stagHunt_dataf1 = pd.read_csv('Data/Experiment1/StagHunt_test_DELETE_RPST_5_1_0_3.csv', index_col=0)

#### _The System ID_
Since these experiments are with unique players, each system is denoted with a unique ID that is comprised of numbers, commas, and underscores. The numbers represent players and determined by the alphabetical order of the player's strategy out of the total set of player strategies in system. The commas delimit players of the same team, and underscores delimit different teams of the same system. Note the system IDs and team compositions below and how they differ between experiment 1 and experiment 2. 

In [6]:
# I use option_context to customize the limits of the dataframe display
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.max_colwidth', None):  # more options can be specified also
    display(classic_dataf1[['System ID', 'Team1', 'Team1 Avg Score',
       'Team1 Min Score', 'Team1 Avg CC Dist']])

,System ID,Team1,Team1 Avg Score,Team1 Min Score,Team1 Avg CC Dist
1,"1,2,3,4,5,6,7,8,9,10","Alternator,Anti Tit For Tat,Bully,Cooperator,Cycler DC,Defector,Suspicious Tit For Tat,Tit For Tat,Win-Shift Lose-Stay,Win-Stay Lose-Shift",2.284385,1.409722,0.248897


In [7]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.max_colwidth', None):  # more options can be specified also
    display(stagHunt_dataf1[['System ID', 'Team1', 'Team1 Avg Score',
       'Team1 Min Score', 'Team1 Avg CC Dist']])

,System ID,Team1,Team1 Avg Score,Team1 Min Score,Team1 Avg CC Dist
1,"1,2,3,4,5,6,7,8,9,10","Alternator,Anti Tit For Tat,Bully,Cooperator,Cycler DC,Defector,Suspicious Tit For Tat,Tit For Tat,Win-Shift Lose-Stay,Win-Stay Lose-Shift",2.257786,1.994048,0.248897


In [8]:
# SIDENOTE: Documentation (i.e. docstring) for code can be retrieved with '?'. Here are some examples.

?pd_exp

Type:        module
String form: <module 'pd_exp' from '/home/s/PrisonersDilemma-main/pd_exp.py'>
File:        ~/PrisonersDilemma-main/pd_exp.py
Docstring:  
Pd_exp: Prisoner's Dilemma Experimentation

Generate data for numerous Prisoner's Dilemma tournaments or systems of 
tournaments.

Classes:

    PdTournament
        A class to represent a tournament.
    PdSystem 
        Generate data for multiple tournaments and organize it into a dataframe
    PdExp
        Generate data for multiple systems
        
Functions:

    grouper(iterable, n, fillvalue=None) -> iterable of n-sized-chunks
        Collect data into fixed-length chunks or blocks
    avg_normalised_state(object, tuple) -> float
        Returns the tournament average for given state distribution (e.g.
        (C,C), (D,D), (C,D), (D,C))


In [9]:
?pd_exp.PdExp

Init signature: pd_exp.PdExp(tuple_of_systems, game_type=None)
Docstring:     
A class to represent a series of systems. 

...

Attributes
----------
sys_tuple : tuple
    three-dimensional ordered list, or tuple, where the first-dimension is
    a list of systems, the 2nd-dimension is the team list, and the 
    3rd-dimension is the individual players.
game : axelrod.game (object)
    container for game matrix and scoring logic 

Methods
-------
run_experiments:
    Iterates through each system, runs the different tournaments and 
    then saves the data to the data attribute.
save_data(path_to_directory, descrip_name):
    Saves experiment data as a csv file
Init docstring:
Constructs all the necessary attributes for pd experiment object

Parameters
----------
tuple_of_systems : tuple
    a multi-dimensional matrix where each item of the first-dimension is an
    ordered list of systems, and the second dimension is an ordered-list of 
    teams
game_type : axelrod.game (object)
    c

### Experiment 2: Comparing systems of 3 teams across classic PD and Stag Hunt

In [10]:
### For experiment 2
# To create numerous systems with a partitioned list of teams can take time, so I save the tuple of systems 
# to file immediately after computing all k-partition subsets of a set of players  to avoid having to recompute 
# them at each run.

# Checking for txt file with list of systems
p = Path('partition_list_teams_of5_DEMO.txt')
if p.exists():
   with open(p, "r") as f:
       part_list = json.load(f)
else:
   print('creating partitions and saving to file')
   part_list = list(partitions(player_names,5))
   p.touch()
   with open(p, "w") as f:
       json.dump(part_list, f)
print(part_list)

[[['Alternator', 'Cycler DC', 'Anti Tit For Tat', 'Win-Stay Lose-Shift', 'Suspicious Tit For Tat'], ['Win-Shift Lose-Stay', 'Tit For Tat', 'Bully', 'Cooperator', 'Defector']], [['Cycler DC', 'Anti Tit For Tat', 'Defector', 'Alternator', 'Suspicious Tit For Tat'], ['Win-Shift Lose-Stay', 'Tit For Tat', 'Bully', 'Cooperator', 'Win-Stay Lose-Shift']], [['Alternator', 'Anti Tit For Tat', 'Defector', 'Win-Stay Lose-Shift', 'Suspicious Tit For Tat'], ['Win-Shift Lose-Stay', 'Tit For Tat', 'Bully', 'Cycler DC', 'Cooperator']], [['Alternator', 'Cycler DC', 'Defector', 'Win-Stay Lose-Shift', 'Suspicious Tit For Tat'], ['Win-Shift Lose-Stay', 'Tit For Tat', 'Bully', 'Cooperator', 'Anti Tit For Tat']], [['Cycler DC', 'Anti Tit For Tat', 'Defector', 'Win-Stay Lose-Shift', 'Suspicious Tit For Tat'], ['Win-Shift Lose-Stay', 'Tit For Tat', 'Bully', 'Cooperator', 'Alternator']], [['Alternator', 'Cycler DC', 'Anti Tit For Tat', 'Defector', 'Win-Stay Lose-Shift'], ['Win-Shift Lose-Stay', 'Tit For Tat', 

In [11]:
part_list = part_list[:3] # Let's just consider 3 different systems
print('Total count of systems: ',len(part_list))
start_time = time.time()

print('Instantiate PD Experiments with different game types')
classic_pdExp = pd_exp.PdExp(part_list)
stag_pdExp = pd_exp.PdExp(part_list,game_type=stag)

print('Running experiments and computing data')
classic_pdExp.run_experiments()
stag_pdExp.run_experiments()

print('Save the data')
path = 'Data/Experiment2/'
classic_pdExp.save_data(path, 'ClassicPD_4x3_12uniq_DEMO_RPST_3_1_0_5')  # CHANGE file_name string when necessary
stag_pdExp.save_data(path, 'StagHunt_4x3_12uniq_DEMO_RPST_5_1_0_3')  # CHANGE file_name string when necessary

print("--- %s seconds ---" % (time.time() - start_time))

Total count of systems:  3
Instantiate PD Experiments with different game types
Running experiments and computing data
partition list:  [['Alternator', 'Cycler DC', 'Anti Tit For Tat', 'Win-Stay Lose-Shift', 'Suspicious Tit For Tat'], ['Win-Shift Lose-Stay', 'Tit For Tat', 'Bully', 'Cooperator', 'Defector']]
Instantiating tournament object with these players:  Alternator,Anti Tit For Tat,Cycler DC,Suspicious Tit For Tat,Win-Stay Lose-Shift
Analysing: 100%|██████████| 25/25 [00:00<00:00, 66.17it/s]
Match length for each user against other users:  [[[3.0, 4.0, 8.0, 21.0, 12.0], [4.0, 20.0, 30.0, 2.0, 4.0], [8.0, 30.0, 6.0, 2.0, 10.0], [21.0, 2.0, 2.0, 16.0, 3.0], [12.0, 4.0, 10.0, 3.0, 6.0]]]
Counting the number of CC states:
player  1 states:
{}
{(C, D): 2, (D, C): 2}
{(C, C): 4, (D, D): 4}
{(C, C): 6, (C, D): 5, (D, C): 5, (D, D): 5}
{(C, D): 6, (D, C): 6}
player  2 states:
{(C, D): 2, (D, C): 2}
{}
{(C, C): 15, (D, C): 1, (D, D): 14}
{(C, D): 1, (D, C): 1}
{(C, D): 2, (D, C): 1, (D, D

### Experiment 2: Data Comparison Across Systems, Teams, and Games

In [12]:
# Let's compare the results
# Retrieve the stored data
classic_dataf = pd.read_csv('Data/Experiment2/ClassicPD_4x3_12uniq_DEMO_RPST_3_1_0_5.csv', index_col=0)
stagHunt_dataf = pd.read_csv('Data/Experiment2/StagHunt_4x3_12uniq_DEMO_RPST_5_1_0_3.csv', index_col=0)

# Reset the index to accurately count the total systems
classic_dataf = classic_dataf.reset_index(drop=True)
stagHunt_dataf = stagHunt_dataf.reset_index(drop=True)

FileNotFoundError: [Errno 2] No such file or directory: 'Data/Experiment2/ClassicPD_4x3_12uniq_DEMO_RPST_3_1_0_5.csv'

In [ ]:
# The computed results create a large spreadsheet or dataframe. Here is a list of just the column names.
classic_dataf.columns

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.max_colwidth', None):  # more options can be specified also
    display(classic_dataf[['System ID', 'Team1', 'Team2']])

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.max_colwidth', None):  # more options can be specified also
    display(stagHunt_dataf[['System ID', 'Team1', 'Team2']])

#### How do the systems compare within the same game?

In [ ]:

classic_dataf.loc[:,:'SYS New CC Dist MIN']

In [ ]:

stagHunt_dataf.loc[:,:'SYS New CC Dist MIN']

#### How did team 1 and 2 performance differ across systems?

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.max_colwidth', None):  # more options can be specified also
    display(classic_dataf[['System ID', 'Team1', 'Team1 Avg Score', 'Team1 New Avg Score', 
       'Team1 Min Score', 'Team1 Avg CC Dist', 'Team1 New Avg CC Dist', 'Team2','Team2 Avg Score', 'Team2 New Avg Score', 
       'Team2 Min Score', 'Team2 Avg CC Dist', 'Team2 New Avg CC Dist']])

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 
                       'display.max_colwidth', None):  # more options can be 
                                                       # specified also
    display(stagHunt_dataf[['System ID', 'Team1', 'Team1 Avg Score', 'Team1 New Avg Score', 
       'Team1 Min Score', 'Team1 Avg CC Dist', 'Team1 New Avg CC Dist', 'Team2','Team2 Avg Score', 'Team2 New Avg Score', 
       'Team2 Min Score', 'Team2 Avg CC Dist', 'Team2 New Avg CC Dist']])

### Experiment 7C: All Cooperators: One team

In [57]:

import axelrod as axl
players = ['Bitter Cooperator', 'Cooperator', 'Fourteen Coop', 'Grim Trigger', 'Thirteen Coop', 'Tit For Tat', 'Win-Stay Lose-Shift']
part_list = list(partitions(players,7))
p = Path('all_7_coop.txt')
p.touch()
with open(p, "w") as f:
    json.dump(part_list, f)


In [58]:
# part_list = part_list[:3] # Let's just consider 3 different teams
# part_list = [[('Cooperator', 'Defector'), ('Cooperator', 'Cooperator'), ('Defector', 'Defector')]]
print('Total count of teams: ',len(part_list))
print(part_list)
start_time = time.time()

print('Instantiate PD Experiments with different game types')
classic_pdExp = pd_exp.PdExp(part_list)
# stag_pdExp = pd_exp.PdExp(part_list,game_type=stag)

print('Running experiments and computing data')
classic_pdExp.run_experiments()
# stag_pdExp.run_experiments()

print('Save the data')
path = 'Data/Experiment2/'
classic_pdExp.save_data(path, 'ClassicPD_7C')  # CHANGE file_name string when necessary
# stag_pdExp.save_data(path, 'StagHunt_4x3_12uniq_DEMO_RPST_5_1_0_3')  # CHANGE file_name string when necessary

print("--- %s seconds ---" % (time.time() - start_time))

Total count of teams:  1
[[('Cooperator', 'Win-Stay Lose-Shift', 'Grim Trigger', 'Tit For Tat', 'Bitter Cooperator', 'Thirteen Coop', 'Fourteen Coop')]]
Instantiate PD Experiments with different game types
Running experiments and computing data
partition list:  [('Cooperator', 'Win-Stay Lose-Shift', 'Grim Trigger', 'Tit For Tat', 'Bitter Cooperator', 'Thirteen Coop', 'Fourteen Coop')]
Instantiating tournament object with these players:  Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player
Analysing: 100%|██████████| 25/25 [00:00<00:00, 109.22it/s]
Match length for each user against other users:  [[[3.0, 4.0, 8.0, 21.0, 12.0, 20.0, 30.0], [4.0, 2.0, 4.0, 6.0, 2.0, 10.0, 16.0], [8.0, 4.0, 3.0, 6.0, 2.0, 12.0, 14.0], [21.0, 6.0, 6.0, 21.0, 8.0, 6.0, 4.0], [12.0, 2.0, 2.0, 8.0, 22.0, 12.0, 9.0], [20.0, 10.0, 12.0, 6.0, 12.0, 17.0, 4.0], [30.0, 16.0, 14.0, 4.0, 9.

### Experiment 7C: Comparing the results

In [59]:
# Let's compare the results
# Retrieve the stored data
classic_dataf = pd.read_csv('Data/Experiment2/ClassicPD_7C_RPST_3_1_0_5.csv', index_col=0)

# Reset the index to accurately count the total systems
classic_dataf = classic_dataf.reset_index(drop=True)

In [60]:

classic_dataf.loc[:,:'SYS New CC Dist MIN']

,System ID,SYS MIN Score,SYS AVG Score,MIN of Team Avgs,AVG of Team Mins,SYS CC Dist AVG,SYS CC Dist MIN,SYS New CC Dist AVG,SYS New CC Dist MIN
0,"2,7,4,6,1,5,3",3.0,3.0,3.0,3.0,1.0,1.0,1.0,1.0


In [61]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.max_colwidth', None):  # more options can be specified also
    display(classic_dataf[['System ID', 'Team1', 'Team1 Avg Score', 'Team1 New Avg Score', 
       'Team1 Min Score', 'Team1 Avg CC Dist', 'Team1 New Avg CC Dist']])

,System ID,Team1,Team1 Avg Score,Team1 New Avg Score,Team1 Min Score,Team1 Avg CC Dist,Team1 New Avg CC Dist
0,"2,7,4,6,1,5,3","Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player",3.0,6.0,3.0,1.0,1.0


### Experiment 7D: All Defectors: One team

In [53]:

import axelrod as axl
from axelrod.action import Action

players = ['Defector', 'Fourteen Defect', 'Stubborn Defect', 'Suspicious Tit For Tat', 'Sucker Defect', 'Two Defect', 'Win-Shift Lose-Stay']
part_list = list(partitions(players,7))

p = Path('all_7_def.txt')
p.touch()
with open(p, "w") as f:
    json.dump(part_list, f)


In [54]:
print('Total count of teams: ',len(part_list))
print(part_list)
start_time = time.time()

print('Instantiate PD Experiments with different game types')
classic_pdExp = pd_exp.PdExp(part_list)

print('Running experiments and computing data')
classic_pdExp.run_experiments()

print('Save the data')
path = 'Data/Experiment2/'
classic_pdExp.save_data(path, 'ClassicPD_7D')  # CHANGE file_name string when necessary

print("--- %s seconds ---" % (time.time() - start_time))

Total count of teams:  1
[[('Stubborn Defect', 'Defector', 'Sucker Defect', 'Fourteen Defect', 'Two Defect', 'Win-Shift Lose-Stay', 'Suspicious Tit For Tat')]]
Instantiate PD Experiments with different game types
Running experiments and computing data
partition list:  [('Stubborn Defect', 'Defector', 'Sucker Defect', 'Fourteen Defect', 'Two Defect', 'Win-Shift Lose-Stay', 'Suspicious Tit For Tat')]
Instantiating tournament object with these players:  Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player
Analysing: 100%|██████████| 25/25 [00:00<00:00, 50.25it/s]

Match length for each user against other users:  [[[3.0, 4.0, 8.0, 21.0, 12.0, 20.0, 30.0], [4.0, 2.0, 4.0, 6.0, 2.0, 10.0, 16.0], [8.0, 4.0, 3.0, 6.0, 2.0, 12.0, 14.0], [21.0, 6.0, 6.0, 21.0, 8.0, 6.0, 4.0], [12.0, 2.0, 2.0, 8.0, 22.0, 12.0, 9.0], [20.0, 10.0, 12.0, 6.0, 12.0, 17.0, 4.0], [30.0, 16.0,

### Experiment 7D: Comparing the results

In [55]:
# Let's compare the results
# Retrieve the stored data
classic_dataf = pd.read_csv('Data/Experiment2/ClassicPD_7D_RPST_3_1_0_5.csv', index_col=0)

# Reset the index to accurately count the total systems
classic_dataf = classic_dataf.reset_index(drop=True)
classic_dataf.loc[:,:'SYS New CC Dist MIN']

,System ID,SYS MIN Score,SYS AVG Score,MIN of Team Avgs,AVG of Team Mins,SYS CC Dist AVG,SYS CC Dist MIN,SYS New CC Dist AVG,SYS New CC Dist MIN
0,"10,8,12,9,13,14,11",1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0


In [56]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.max_colwidth', None):  # more options can be specified also
    display(classic_dataf[['System ID', 'Team1', 'Team1 Avg Score', 'Team1 New Avg Score', 
       'Team1 Min Score', 'Team1 Avg CC Dist', 'Team1 New Avg CC Dist']])

,System ID,Team1,Team1 Avg Score,Team1 New Avg Score,Team1 Min Score,Team1 Avg CC Dist,Team1 New Avg CC Dist
0,"10,8,12,9,13,14,11","Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player",1.0,2.0,1.0,0.0,0.0


### Experiment 7C1D: All Cooperators + Defector: One team

In [49]:
import axelrod as axl
players = ['Bitter Cooperator', 'Cooperator', 'Fourteen Coop', 'Grim Trigger', 'Thirteen Coop', 'Tit For Tat', 'Win-Stay Lose-Shift', 'Defector']
part_list = list(partitions(players,8))

p = Path('coop_7_def_1.txt')
p.touch()
with open(p, "w") as f:
    json.dump(part_list, f)

In [50]:
print('Total count of teams: ',len(part_list))
print(part_list)
start_time = time.time()

print('Instantiate PD Experiments with different game types')
classic_pdExp = pd_exp.PdExp(part_list)

print('Running experiments and computing data')
classic_pdExp.run_experiments()

print('Save the data')
path = 'Data/Experiment2/'
classic_pdExp.save_data(path, 'ClassicPD_7C1D')  # CHANGE file_name string when necessary

print("--- %s seconds ---" % (time.time() - start_time))

Total count of teams:  1
[[('Cooperator', 'Win-Stay Lose-Shift', 'Grim Trigger', 'Defector', 'Tit For Tat', 'Bitter Cooperator', 'Thirteen Coop', 'Fourteen Coop')]]
Instantiate PD Experiments with different game types
Running experiments and computing data
partition list:  [('Cooperator', 'Win-Stay Lose-Shift', 'Grim Trigger', 'Defector', 'Tit For Tat', 'Bitter Cooperator', 'Thirteen Coop', 'Fourteen Coop')]
Instantiating tournament object with these players:  Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player
Analysing: 100%|██████████| 25/25 [00:00<00:00, 68.50it/s]
Match length for each user against other users:  [[[3.0, 4.0, 8.0, 21.0, 12.0, 20.0, 30.0, 2.0], [4.0, 4.0, 6.0, 2.0, 10.0, 16.0, 3.0, 6.0], [8.0, 6.0, 2.0, 12.0, 14.0, 21.0, 8.0, 6.0], [21.0, 2.0, 12.0, 4.0, 22.0, 12.0, 9.0, 17.0], [12.0, 10.0, 14.0, 22.0, 4.0, 3.0, 

### Experiment 7C1D: Comparing the results

In [51]:
classic_dataf = pd.read_csv('Data/Experiment2/ClassicPD_7C1D_RPST_3_1_0_5.csv', index_col=0)

classic_dataf = classic_dataf.reset_index(drop=True)
classic_dataf.loc[:,:'SYS New CC Dist MIN']

,System ID,SYS MIN Score,SYS AVG Score,MIN of Team Avgs,AVG of Team Mins,SYS CC Dist AVG,SYS CC Dist MIN,SYS New CC Dist AVG,SYS New CC Dist MIN
0,"2,7,4,8,6,1,5,3",2.571429,2.721185,2.721185,2.571429,0.75,0.75,0.682274,0.682274


In [52]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.max_colwidth', None):  # more options can be specified also
    display(classic_dataf[['System ID', 'Team1', 'Team1 Avg Score', 'Team1 New Avg Score', 
       'Team1 Min Score', 'Team1 Avg CC Dist', 'Team1 New Avg CC Dist']])

,System ID,Team1,Team1 Avg Score,Team1 New Avg Score,Team1 Min Score,Team1 Avg CC Dist,Team1 New Avg CC Dist
0,"2,7,4,8,6,1,5,3","Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player",2.721185,5.29097,2.571429,0.75,0.682274


### Experiment 7C2D: All Cooperators + 2 Defectors: One team

In [45]:
import axelrod as axl
players = ['Bitter Cooperator', 'Cooperator', 'Fourteen Coop', 'Grim Trigger', 'Thirteen Coop', 'Tit For Tat', 'Win-Stay Lose-Shift', 'Defector', 'Fourteen Defect']
part_list = list(partitions(players,9))

p = Path('coop_7_def_2.txt')
p.touch()
with open(p, "w") as f:
    json.dump(part_list, f)

In [46]:
print('Total count of teams: ',len(part_list))
print(part_list)
start_time = time.time()

print('Instantiate PD Experiments with different game types')
classic_pdExp = pd_exp.PdExp(part_list)

print('Running experiments and computing data')
classic_pdExp.run_experiments()

print('Save the data')
path = 'Data/Experiment2/'
classic_pdExp.save_data(path, 'ClassicPD_7C2D')  # CHANGE file_name string when necessary

print("--- %s seconds ---" % (time.time() - start_time))

Total count of teams:  1
[[('Grim Trigger', 'Fourteen Defect', 'Tit For Tat', 'Bitter Cooperator', 'Thirteen Coop', 'Cooperator', 'Defector', 'Win-Stay Lose-Shift', 'Fourteen Coop')]]
Instantiate PD Experiments with different game types
Running experiments and computing data
partition list:  [('Grim Trigger', 'Fourteen Defect', 'Tit For Tat', 'Bitter Cooperator', 'Thirteen Coop', 'Cooperator', 'Defector', 'Win-Stay Lose-Shift', 'Fourteen Coop')]
Instantiating tournament object with these players:  Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player
Analysing: 100%|██████████| 25/25 [00:00<00:00, 56.96it/s]
Match length for each user against other users:  [[[3.0, 4.0, 8.0, 21.0, 12.0, 20.0, 30.0, 2.0, 4.0], [4.0, 6.0, 2.0, 10.0, 16.0, 3.0, 6.0, 2.0, 12.0], [8.0, 2.0, 14.0, 21.0, 8.0, 6.0, 4.0, 22.0, 12.0], [

### Experiment 7C2D: Comparing the results

In [47]:
classic_dataf = pd.read_csv('Data/Experiment2/ClassicPD_7C2D_RPST_3_1_0_5.csv', index_col=0)

classic_dataf = classic_dataf.reset_index(drop=True)
classic_dataf.loc[:,:'SYS New CC Dist MIN']

,System ID,SYS MIN Score,SYS AVG Score,MIN of Team Avgs,AVG of Team Mins,SYS CC Dist AVG,SYS CC Dist MIN,SYS New CC Dist AVG,SYS New CC Dist MIN
0,"4,9,6,1,5,2,8,7,3",2.5,2.689873,2.689873,2.5,0.675579,0.675579,0.697548,0.697548


In [48]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.max_colwidth', None):  # more options can be specified also
    display(classic_dataf[['System ID', 'Team1', 'Team1 Avg Score', 'Team1 New Avg Score', 
       'Team1 Min Score', 'Team1 Avg CC Dist', 'Team1 New Avg CC Dist']])

,System ID,Team1,Team1 Avg Score,Team1 New Avg Score,Team1 Min Score,Team1 Avg CC Dist,Team1 New Avg CC Dist
0,"4,9,6,1,5,2,8,7,3","Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player",2.689873,5.297003,2.5,0.675579,0.697548


### Experiment 7C3D: All Cooperators + 3 Defectors: One team

In [41]:
import axelrod as axl
players = ['Bitter Cooperator', 'Cooperator', 'Fourteen Coop', 'Grim Trigger', 'Thirteen Coop', 'Tit For Tat', 'Win-Stay Lose-Shift', 'Defector', 'Fourteen Defect', 'Stubborn Defect']
part_list = list(partitions(players,10))

p = Path('coop_7_def_3.txt')
p.touch()
with open(p, "w") as f:
    json.dump(part_list, f)

In [42]:
print('Total count of teams: ',len(part_list))
print(part_list)
start_time = time.time()

print('Instantiate PD Experiments with different game types')
classic_pdExp = pd_exp.PdExp(part_list)

print('Running experiments and computing data')
classic_pdExp.run_experiments()

print('Save the data')
path = 'Data/Experiment2/'
classic_pdExp.save_data(path, 'ClassicPD_7C3D')  # CHANGE file_name string when necessary

print("--- %s seconds ---" % (time.time() - start_time))

Total count of teams:  1
[[('Grim Trigger', 'Fourteen Defect', 'Bitter Cooperator', 'Tit For Tat', 'Thirteen Coop', 'Cooperator', 'Stubborn Defect', 'Win-Stay Lose-Shift', 'Defector', 'Fourteen Coop')]]
Instantiate PD Experiments with different game types
Running experiments and computing data
partition list:  [('Grim Trigger', 'Fourteen Defect', 'Bitter Cooperator', 'Tit For Tat', 'Thirteen Coop', 'Cooperator', 'Stubborn Defect', 'Win-Stay Lose-Shift', 'Defector', 'Fourteen Coop')]
Instantiating tournament object with these players:  Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player
Analysing: 100%|██████████| 25/25 [00:00<00:00, 54.30it/s]

Match length for each user against other users:  [[[3.0, 4.0, 8.0, 21.0, 12.0, 20.0, 30.0, 2.0, 4.0, 6.0], [4.0, 2.0, 10.0, 16.0, 3.0, 6.0,

### Experiment 7C3D: Comparing the results

In [43]:
classic_dataf = pd.read_csv('Data/Experiment2/ClassicPD_7C3D_RPST_3_1_0_5.csv', index_col=0)

classic_dataf = classic_dataf.reset_index(drop=True)
classic_dataf.loc[:,:'SYS New CC Dist MIN']

,System ID,SYS MIN Score,SYS AVG Score,MIN of Team Avgs,AVG of Team Mins,SYS CC Dist AVG,SYS CC Dist MIN,SYS New CC Dist AVG,SYS New CC Dist MIN
0,"4,9,1,6,5,2,10,7,8,3",2.177712,2.505347,2.505347,2.177712,0.558386,0.558386,0.519669,0.519669


In [44]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.max_colwidth', None):  # more options can be specified also
    display(classic_dataf[['System ID', 'Team1', 'Team1 Avg Score', 'Team1 New Avg Score', 
       'Team1 Min Score', 'Team1 Avg CC Dist', 'Team1 New Avg CC Dist']])

,System ID,Team1,Team1 Avg Score,Team1 New Avg Score,Team1 Min Score,Team1 Avg CC Dist,Team1 New Avg CC Dist
0,"4,9,1,6,5,2,10,7,8,3","Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player",2.505347,4.749482,2.177712,0.558386,0.519669


### Experiment 7C4D: All Cooperators + 4 Defectors: One team

In [37]:
import axelrod as axl
players = ['Bitter Cooperator', 'Cooperator', 'Fourteen Coop', 'Grim Trigger', 'Thirteen Coop', 'Tit For Tat', 'Win-Stay Lose-Shift', 'Defector', 'Fourteen Defect', 'Stubborn Defect', 'Suspicious Tit For Tat']
part_list = list(partitions(players,11))

p = Path('coop_7_def_4.txt')
p.touch()
with open(p, "w") as f:
    json.dump(part_list, f)

In [38]:
print('Total count of teams: ',len(part_list))
print(part_list)
start_time = time.time()

print('Instantiate PD Experiments with different game types')
classic_pdExp = pd_exp.PdExp(part_list)

print('Running experiments and computing data')
classic_pdExp.run_experiments()

print('Save the data')
path = 'Data/Experiment2/'
classic_pdExp.save_data(path, 'ClassicPD_7C4D')  # CHANGE file_name string when necessary

print("--- %s seconds ---" % (time.time() - start_time))

Total count of teams:  1
[[('Grim Trigger', 'Fourteen Defect', 'Tit For Tat', 'Bitter Cooperator', 'Thirteen Coop', 'Cooperator', 'Stubborn Defect', 'Win-Stay Lose-Shift', 'Defector', 'Suspicious Tit For Tat', 'Fourteen Coop')]]
Instantiate PD Experiments with different game types
Running experiments and computing data
partition list:  [('Grim Trigger', 'Fourteen Defect', 'Tit For Tat', 'Bitter Cooperator', 'Thirteen Coop', 'Cooperator', 'Stubborn Defect', 'Win-Stay Lose-Shift', 'Defector', 'Suspicious Tit For Tat', 'Fourteen Coop')]
Instantiating tournament object with these players:  Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player
Analysing: 100%|██████████| 25/25 [00:00<00:00, 48.44it/s]

Match length for each user against other users:  [[[3.0, 4.0,

### Experiment 7C4D: Comparing the results

In [39]:
classic_dataf = pd.read_csv('Data/Experiment2/ClassicPD_7C4D_RPST_3_1_0_5.csv', index_col=0)

classic_dataf = classic_dataf.reset_index(drop=True)
classic_dataf.loc[:,:'SYS New CC Dist MIN']

,System ID,SYS MIN Score,SYS AVG Score,MIN of Team Avgs,AVG of Team Mins,SYS CC Dist AVG,SYS CC Dist MIN,SYS New CC Dist AVG,SYS New CC Dist MIN
0,"4,9,6,1,5,2,10,7,8,11,3",2.087798,2.409297,2.409297,2.087798,0.489442,0.489442,0.553903,0.553903


In [40]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.max_colwidth', None):  # more options can be specified also
    display(classic_dataf[['System ID', 'Team1', 'Team1 Avg Score', 'Team1 New Avg Score', 
       'Team1 Min Score', 'Team1 Avg CC Dist', 'Team1 New Avg CC Dist']])

,System ID,Team1,Team1 Avg Score,Team1 New Avg Score,Team1 Min Score,Team1 Avg CC Dist,Team1 New Avg CC Dist
0,"4,9,6,1,5,2,10,7,8,11,3","Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player",2.409297,4.840149,2.087798,0.489442,0.553903


### Experiment 7C5D: All Cooperators + 5 Defectors: One team


In [33]:
import axelrod as axl
players = ['Bitter Cooperator', 'Cooperator', 'Fourteen Coop', 'Grim Trigger', 'Thirteen Coop', 'Tit For Tat', 'Win-Stay Lose-Shift', 'Defector', 'Fourteen Defect', 'Stubborn Defect', 'Suspicious Tit For Tat', 'Sucker Defect']
part_list = list(partitions(players,12))

p = Path('coop_7_def_5.txt')
p.touch()
with open(p, "w") as f:
    json.dump(part_list, f)

In [34]:
print('Total count of teams: ',len(part_list))
print(part_list)
start_time = time.time()

print('Instantiate PD Experiments with different game types')
classic_pdExp = pd_exp.PdExp(part_list)

print('Running experiments and computing data')
classic_pdExp.run_experiments()

print('Save the data')
path = 'Data/Experiment2/'
classic_pdExp.save_data(path, 'ClassicPD_7C5D')  # CHANGE file_name string when necessary

print("--- %s seconds ---" % (time.time() - start_time))

Total count of teams:  1
[[('Grim Trigger', 'Sucker Defect', 'Fourteen Defect', 'Bitter Cooperator', 'Tit For Tat', 'Thirteen Coop', 'Cooperator', 'Stubborn Defect', 'Win-Stay Lose-Shift', 'Defector', 'Suspicious Tit For Tat', 'Fourteen Coop')]]
Instantiate PD Experiments with different game types
Running experiments and computing data
partition list:  [('Grim Trigger', 'Sucker Defect', 'Fourteen Defect', 'Bitter Cooperator', 'Tit For Tat', 'Thirteen Coop', 'Cooperator', 'Stubborn Defect', 'Win-Stay Lose-Shift', 'Defector', 'Suspicious Tit For Tat', 'Fourteen Coop')]
Instantiating tournament object with these players:  Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player
Analysing: 100%|██████████| 25/25 [00:00<00:00, 48.16it/s]

M

### Experiment 7C5D: Comparing the results

In [35]:
classic_dataf = pd.read_csv('Data/Experiment2/ClassicPD_7C5D_RPST_3_1_0_5.csv', index_col=0)

classic_dataf = classic_dataf.reset_index(drop=True)
classic_dataf.loc[:,:'SYS New CC Dist MIN']

,System ID,SYS MIN Score,SYS AVG Score,MIN of Team Avgs,AVG of Team Mins,SYS CC Dist AVG,SYS CC Dist MIN,SYS New CC Dist AVG,SYS New CC Dist MIN
0,"4,12,9,1,6,5,2,10,7,8,11,3",1.769231,2.317658,2.317658,1.769231,0.418543,0.418543,0.493528,0.493528


In [36]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.max_colwidth', None):  # more options can be specified also
    display(classic_dataf[['System ID', 'Team1', 'Team1 Avg Score', 'Team1 New Avg Score', 
       'Team1 Min Score', 'Team1 Avg CC Dist', 'Team1 New Avg CC Dist']])

,System ID,Team1,Team1 Avg Score,Team1 New Avg Score,Team1 Min Score,Team1 Avg CC Dist,Team1 New Avg CC Dist
0,"4,12,9,1,6,5,2,10,7,8,11,3","Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player",2.317658,4.81877,1.769231,0.418543,0.493528


### Experiment 7C6D: All Cooperators + 5 Defectors: One team


In [29]:
import axelrod as axl
players = ['Bitter Cooperator', 'Cooperator', 'Fourteen Coop', 'Grim Trigger', 'Thirteen Coop', 'Tit For Tat', 'Win-Stay Lose-Shift', 'Defector', 'Fourteen Defect', 'Stubborn Defect', 'Suspicious Tit For Tat', 'Sucker Defect', 'Two Defect']
part_list = list(partitions(players,13))

p = Path('coop_7_def_6.txt')
p.touch()
with open(p, "w") as f:
    json.dump(part_list, f)

In [30]:
print('Total count of teams: ',len(part_list))
print(part_list)
start_time = time.time()

print('Instantiate PD Experiments with different game types')
classic_pdExp = pd_exp.PdExp(part_list)

print('Running experiments and computing data')
classic_pdExp.run_experiments()

print('Save the data')
path = 'Data/Experiment2/'
classic_pdExp.save_data(path, 'ClassicPD_7C6D')  # CHANGE file_name string when necessary

print("--- %s seconds ---" % (time.time() - start_time))

Total count of teams:  1
[[('Grim Trigger', 'Sucker Defect', 'Fourteen Defect', 'Tit For Tat', 'Bitter Cooperator', 'Two Defect', 'Thirteen Coop', 'Cooperator', 'Stubborn Defect', 'Win-Stay Lose-Shift', 'Defector', 'Suspicious Tit For Tat', 'Fourteen Coop')]]
Instantiate PD Experiments with different game types
Running experiments and computing data
partition list:  [('Grim Trigger', 'Sucker Defect', 'Fourteen Defect', 'Tit For Tat', 'Bitter Cooperator', 'Two Defect', 'Thirteen Coop', 'Cooperator', 'Stubborn Defect', 'Win-Stay Lose-Shift', 'Defector', 'Suspicious Tit For Tat', 'Fourteen Coop')]
Instantiating tournament object with these players:  Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player
Analysi

### Experiment 7C6D: Comparing the results

In [31]:
classic_dataf = pd.read_csv('Data/Experiment2/ClassicPD_7C6D_RPST_3_1_0_5.csv', index_col=0)

classic_dataf = classic_dataf.reset_index(drop=True)
classic_dataf.loc[:,:'SYS New CC Dist MIN']

,System ID,SYS MIN Score,SYS AVG Score,MIN of Team Avgs,AVG of Team Mins,SYS CC Dist AVG,SYS CC Dist MIN,SYS New CC Dist AVG,SYS New CC Dist MIN
0,"4,12,9,6,1,13,5,2,10,7,8,11,3",1.75,2.195015,2.195015,1.75,0.330128,0.330128,0.33011,0.33011


In [32]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.max_colwidth', None):  # more options can be specified also
    display(classic_dataf[['System ID', 'Team1', 'Team1 Avg Score', 'Team1 New Avg Score', 
       'Team1 Min Score', 'Team1 Avg CC Dist', 'Team1 New Avg CC Dist']])

,System ID,Team1,Team1 Avg Score,Team1 New Avg Score,Team1 Min Score,Team1 Avg CC Dist,Team1 New Avg CC Dist
0,"4,12,9,6,1,13,5,2,10,7,8,11,3","Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player",2.195015,4.44337,1.75,0.330128,0.33011


### Experiment 7C7D: All Cooperators + 5 Defectors: One team


In [25]:
import axelrod as axl
players = ['Bitter Cooperator', 'Cooperator', 'Fourteen Coop', 'Grim Trigger', 'Thirteen Coop', 'Tit For Tat', 'Win-Stay Lose-Shift', 'Defector', 'Fourteen Defect', 'Stubborn Defect', 'Suspicious Tit For Tat', 'Sucker Defect', 'Two Defect', 'Win-Shift Lose-Stay']
part_list = list(partitions(players,14))

p = Path('coop_7_def_7.txt')
p.touch()
with open(p, "w") as f:
    json.dump(part_list, f)

In [26]:
print('Total count of teams: ',len(part_list))
print(part_list)
start_time = time.time()

print('Instantiate PD Experiments with different game types')
classic_pdExp = pd_exp.PdExp(part_list)

print('Running experiments and computing data')
classic_pdExp.run_experiments()

print('Save the data')
path = 'Data/Experiment2/'
classic_pdExp.save_data(path, 'ClassicPD_7C7D')  # CHANGE file_name string when necessary

print("--- %s seconds ---" % (time.time() - start_time))

Total count of teams:  1
[[('Grim Trigger', 'Sucker Defect', 'Fourteen Defect', 'Bitter Cooperator', 'Tit For Tat', 'Two Defect', 'Thirteen Coop', 'Win-Shift Lose-Stay', 'Cooperator', 'Stubborn Defect', 'Win-Stay Lose-Shift', 'Defector', 'Suspicious Tit For Tat', 'Fourteen Coop')]]
Instantiate PD Experiments with different game types
Running experiments and computing data
partition list:  [('Grim Trigger', 'Sucker Defect', 'Fourteen Defect', 'Bitter Cooperator', 'Tit For Tat', 'Two Defect', 'Thirteen Coop', 'Win-Shift Lose-Stay', 'Cooperator', 'Stubborn Defect', 'Win-Stay Lose-Shift', 'Defector', 'Suspicious Tit For Tat', 'Fourteen Coop')]
Instantiating tournament object with these players:  Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memor

### Experiment 7C7D: Comparing the results

In [27]:
classic_dataf = pd.read_csv('Data/Experiment2/ClassicPD_7C7D_RPST_3_1_0_5.csv', index_col=0)

classic_dataf = classic_dataf.reset_index(drop=True)
classic_dataf.loc[:,:'SYS New CC Dist MIN']

,System ID,SYS MIN Score,SYS AVG Score,MIN of Team Avgs,AVG of Team Mins,SYS CC Dist AVG,SYS CC Dist MIN,SYS New CC Dist AVG,SYS New CC Dist MIN
0,"4,12,9,1,6,13,5,14,2,10,7,8,11,3",1.777972,2.140287,2.140287,1.777972,0.304004,0.304004,0.316364,0.316364


In [28]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.max_colwidth', None):  # more options can be specified also
    display(classic_dataf[['System ID', 'Team1', 'Team1 Avg Score', 'Team1 New Avg Score', 
       'Team1 Min Score', 'Team1 Avg CC Dist', 'Team1 New Avg CC Dist']])

,System ID,Team1,Team1 Avg Score,Team1 New Avg Score,Team1 Min Score,Team1 Avg CC Dist,Team1 New Avg CC Dist
0,"4,12,9,1,6,13,5,14,2,10,7,8,11,3","Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player,Generic Memory One Player",2.140287,4.112727,1.777972,0.304004,0.316364


## Total Results

In [63]:
classic_dataf = pd.read_csv('Data/Experiment2/Total.csv', index_col=0)

classic_dataf = classic_dataf.reset_index(drop=True)
classic_dataf.loc[:,:'SYS New CC Dist MIN']

,System ID,SYS MIN Score,SYS AVG Score,MIN of Team Avgs,AVG of Team Mins,SYS CC Dist AVG,SYS CC Dist MIN,SYS New CC Dist AVG,SYS New CC Dist MIN
0,7C,3.000000,3.000000,3.000000,3.000000,1.000000,1.000000,1.000000,1.000000
1,7C1D,2.571429,2.721185,2.721185,2.571429,0.750000,0.750000,0.682274,0.682274
2,7C2D,2.500000,2.689873,2.689873,2.500000,0.675579,0.675579,0.697548,0.697548
3,7C3D,2.177712,2.505347,2.505347,2.177712,0.558386,0.558386,0.519669,0.519669
4,7C4D,2.087798,2.409297,2.409297,2.087798,0.489442,0.489442,0.553903,0.553903
5,7C5D,1.769231,2.317658,2.317658,1.769231,0.418543,0.418543,0.493528,0.493528
6,7C6D,1.750000,2.195015,2.195015,1.750000,0.330128,0.330128,0.330110,0.330110
7,7C7D,1.777972,2.140287,2.140287,1.777972,0.304004,0.304004,0.316364,0.316364
8,7D,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000
